In [1]:
# Make sure we have the packages we need
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression # package for fitting the regression model

np.random.seed(123456789)

## Least Square Monte Carlo Approach

### Model Inputs

In [2]:
# Assume underlying stock price follows Geometric Brownian Motion(GBM)
# set up parameters
reps = 10000 # number of simulations
T = 5 # American option maturity date (in annum)
dt = 1/244 # time difference factor (244 trading days per annum)
time = np.arange(dt,T+dt,dt) # time points
N = len(time) # number of time intervals
r = 0.06 # riskfree interest rate
sigma = 0.3 # volatility or "randomness"
d = 0.0 # dividend yield
S_0 = 1 # initial stock price
k = 1.1 # strike price of the American option

### Monte Carlo Simulation

In [3]:
# contruct the Geometric Brownian Motion(GBM) by random walk construction
# simulate the underlying price until maturity date T across each discretized time point dt
def underlying_price(reps, N, S_0, r, sigma, dt, d):    
    '''
    Simulate the underlying stock price paths by Geometric Brownian motion with mean "r" and volatility "sigma".

    Args:
        reps: number of simulated underlying price paths generated by Geometric Brownian Motion
        N: number of time intervals until maturity
        S_0: initial stock price
        r: (annualized) riskfree interest rate (mean of the GBM)
        sigma: (annualized) volatility (standard deviation) of the GBM
        dt: time difference factor
        d: (annualized) continuous dividend yield
        
    Return:
        S: the matrix of the underlying stock price paths given in each discretized time point (in numpy.array)
    
    '''
    
    S = np.zeros([reps,N+1]) # 'reps' sample paths of GBM(r,sigma) movement

    for i in range(reps):
            
        one_path = S_0*np.cumprod(np.exp((r-d-(sigma**2)/2)*dt + sigma*np.sqrt(dt)*np.random.normal(0,1,N))) # one GBM increment
        
        one_path = np.insert(one_path,0,S_0) # add the initial stock price at inception
        
        S[i,] = one_path

    return S

### Least Square Monte Carlo Approach (2001) by Longstaff  and Schwartz

In [4]:
# approximate each path's discounted conditional expectation value by least square regression
# inspired by Longstaff-Schwartz(2001) Least Sqaure Monte Carlo approach
# a simple LSM approach for pricing American option
def least_square_monte_carlo(reps, N, k, r, sigma, dt, S_0, d, optionType="c", S=None):   
    '''
    Inspired by the Longstaff-Schwartz(2001) Least Sqaure Monte Carlo approach, this function builds a simple and 
    generalized pricing framework for the American option by working in backwardation. 
    
    Args:
        reps: number of simulated underlying stock paths generated by Geometric Brownian Motion
        N: number of time intervals until maturity
        K: strike price for American option
        r: (annualized) continuous riskfree interest rate
        sigma: (annualized) volatility (standard deviation)
        S_0: initial stock price
        d: (annualized) continuous dividend yield
        optionType: if input is "c", solving for American call option; if input is "p", solving for American put option
        S: simulated paths generated by Geometric Brownian Motion (in numpy.array)
        
    Return:
        value: the value (cash flow) matrix of the American option given in each discretized time point (in numpy.array)
    
    '''
    
    def underlying_price(reps, N, S_0, r, sigma, dt):    

        S = S_0*np.cumprod(np.exp((r-d-(sigma**2)/2)*dt + sigma*np.sqrt(dt)*np.random.normal(0,1,(reps,N))), axis = 1) # one GBM increment

        S = np.insert(S,0,S_0, axis = 1) # add the initial stock price at inception

        return S
    
    if S is None: # if user didn't input stock price matrix
    
        S = underlying_price(reps, N, S_0, r, sigma, dt) # obtain the MC simulated underlying price
    
    value = np.zeros([reps,N+1]) # final optimal American option cash flow matrix
    
    for i in range(N,-1,-1):
        
        S_current = S[:,i] # current stock price across all paths
        
        if i == N: # determine if exercise the option at maturity
            
            if optionType == "c":
                
                value[:,i] = np.abs((S_current - k)*(S_current > k)) # store the American call option payoff if greater than 0
                
            elif optionType == "p":
            
                value[:,i] = np.abs((k - S_current)*(k > S_current)) # store the American put option payoff if greater than 0
            
        elif i == 0: # determine the discounted conditional expectation value at inception
            
            value[:,i] = np.exp(-r*dt)*value[:,i+1]
            
        else: # do backwardation until the inception
            
            if optionType == "c":
                
                in_the_money_path_loc = ((S_current - k)*(S_current > k)).nonzero() # determine the in-the-money sample paths location where immediate exercise value > 0 for American call option
            
                exercise_value = ((S_current - k)*(S_current > k))[in_the_money_path_loc] # immediately exercise value at current node for American call option
                
            elif optionType == "p":
                         
                in_the_money_path_loc = ((k - S_current)*(k > S_current)).nonzero() # determine the in-the-money sample paths location where immediate exercise value > 0 for American put option
            
                exercise_value = ((k - S_current)*(k > S_current))[in_the_money_path_loc] # immediately exercise value at current node for American put option
            
            length = len(in_the_money_path_loc[0])
            
            X = np.zeros([length,2]) # the regressor X is the in-the-money path's underlying price at current node
            
            for j in range(1,3):
                
                X[:,j-1] = np.power(S_current[in_the_money_path_loc],j) # our predictor variable X along with the basis function X^2 in LSE regression setup
            
            previous_continuation_value = value[:,i+1][in_the_money_path_loc] # the conditional expectation value in previous node
    
            Y = np.exp(-r*dt)*previous_continuation_value # our target variable Y (discounted value from continuation) in LSE regression setup
            
            try:
            
                least_square_reg_model = LinearRegression().fit(X, Y) # fit the LSE regression to estimate the discounted conditional expectation value
                
            except ValueError:
                
                raise Exception("At the time point (node)", i, "all paths are not optimal to convert into shares")
            
            predicted_conditional_exp = least_square_reg_model.predict(X) # predict the discounted (continued) conditional expectation value
        
            value[:,i] = np.exp(-r*dt)*value[:,i+1]# if do not early exercise the American option, the value of current path will just be the discounted continued option value from the previous node
            
            exercise_immediate_loc = (exercise_value > predicted_conditional_exp) # the optimal early exercise sample path location determined by LSM algorithm 
            
            optimal_loc = in_the_money_path_loc[0][exercise_immediate_loc] # optimal sample paths location at which we should early exercise
            
            optimal_cash_flow = exercise_value[exercise_immediate_loc] # optimal cash flow at current node
            
            value[optimal_loc,(i+1):] = 0 # the early exercise sample paths will have 0 value after the current time point (node) since the option can only be exercised once
            
            value[optimal_loc,i] = optimal_cash_flow # update the current optimal early exercise sample paths' value

    return value

In [5]:
value = least_square_monte_carlo(reps, N, k, r, sigma, dt, S_0, d, optionType = "p")
pd.DataFrame(value) # cash flow matrix of the American put option across time

,0,1,2,3,4,5,6,7,8,9,...,1211,1212,1213,1214,1215,1216,1217,1218,1219,1220
0,0.366501,0.366591,0.366682,0.366772,0.366862,0.366952,0.367042,0.367133,0.367223,0.367313,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.367615,0.367706,0.367796,0.367886,0.367977,0.368067,0.368158,0.368249,0.368339,0.368430,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.353829,0.353916,0.354003,0.354090,0.354177,0.354264,0.354351,0.354438,0.354525,0.354613,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,0.338614,0.338697,0.338781,0.338864,0.338947,0.339031,0.339114,0.339198,0.339281,0.339364,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,0.350047,0.350133,0.350219,0.350305,0.350391,0.350477,0.350564,0.350650,0.350736,0.350822,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Numerical Example Testing

In [6]:
# set up parameters (the American put option example used in paper)
reps = 8 # number of simulations
T = 3 #1 # maturity date
dt = 1 #2**(-10) # time difference factor
time = np.arange(0,T,dt) # time points
N = len(time) # number of time intervals
r = 0.06 # riskfree interest rate
sigma = 0.3 # volatility or "randomness"
d = 0.0 # dividend yield
S_0 = 1 # initial stock price
k = 1.1 # strike price of the American put option

In [7]:
# test random generated underlying price movement (the American put option example used in paper)
S = np.array([[1,1.09,1.08,1.34],
              [1,1.16,1.26,1.54],
              [1,1.22,1.07,1.03],
              [1,0.93,0.97,0.92],
              [1,1.11,1.56,1.52],
              [1,0.76,0.77,0.9],
              [1,0.92,0.84,1.01],
              [1,0.88,1.22,1.34]])
pd.DataFrame(S)

,0,1,2,3
0,1.0,1.09,1.08,1.34
1,1.0,1.16,1.26,1.54
2,1.0,1.22,1.07,1.03
3,1.0,0.93,0.97,0.92
4,1.0,1.11,1.56,1.52
5,1.0,0.76,0.77,0.90
6,1.0,0.92,0.84,1.01
7,1.0,0.88,1.22,1.34


In [8]:
value = least_square_monte_carlo(reps, N, k, r, sigma, dt, S_0, d, optionType = "p", S = S)
pd.DataFrame(value) # cash flow matrix of the American put option across time

,0,1,2,3
0,0.000000,0.000000,0.000000,0.00
1,0.000000,0.000000,0.000000,0.00
2,0.058469,0.062084,0.065924,0.07
3,0.160100,0.170000,0.000000,0.00
4,0.000000,0.000000,0.000000,0.00
5,0.320200,0.340000,0.000000,0.00
6,0.169518,0.180000,0.000000,0.00
7,0.207188,0.220000,0.000000,0.00


In [9]:
np.mean(value[:,0]) # the example American put option price estimated by LSM algorithm

0.11443433004505696

In [10]:
np.max(value[:,0])

0.32019994141864466

In [11]:
# test runtime for the self-defined function
%reload_ext line_profiler
%lprun -f least_square_monte_carlo least_square_monte_carlo(reps, N, k, r, sigma, dt, S_0, d, optionType = "p")

In [ ]:
# # python debugger
# import pdb
# pdb.set_trace()